# Data Visualizer

In [2]:
import sys
sys.path.insert(0, '../')
import AutoSDF
import torch
import os
from AutoSDF.utils.util_3d import init_mesh_renderer, sdf_to_mesh
from AutoSDF.utils.demo_util import get_shape_comp_opt

from AutoSDF.utils.qual_util import save_mesh_as_gif, save_meshes_to_file
from IPython.display import Image
# enable hot reloading
%reload_ext autoreload
%autoreload 2
# Define the options, in this case which GPU we want to use
gpu_id = 0
opt = get_shape_comp_opt(gpu_id=gpu_id)
print(opt.device)
# set up the results directory
res_dir = 'results/results_synthetisch/8e_onder_4/buis_recht'
if not os.path.exists(res_dir): os.mkdir(res_dir)

# Setup the renderer to display gifs
dist, elev, azim = 1.7, 20, 110
mesh_renderer = init_mesh_renderer(image_size=256, dist=dist, elev=elev, azim=azim, device = opt.device)

student
geomatica-linux-jelle
cuda:0


In [3]:
sdfPath = r"/home/student/Documents/Data/Pre-processed_data/windows-preprocessed/synthetisch_0_v2/buis_recht/recht_bocht_45/sdf.pt"
sdf = torch.load(sdfPath,map_location=torch.device(opt.device))
gen_sdf_as_mesh = sdf_to_mesh(sdf) 

# save as gif
gif_name = f'{res_dir}/shape-example.gif'
save_mesh_as_gif(mesh_renderer, gen_sdf_as_mesh, nrow=1, out_name=gif_name, device=opt.device)
Image(gif_name)

(256, 256, 4)
0 255


In [1]:
import trimesh
import pyrender
import numpy as np
import imageio
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"  # or "osmesa"

def render_obj(obj_path, n_frames=36, image_size=(512, 512)):
    obj_path = Path(obj_path)

    # Check if PNG and GIF already exist
    image_path = obj_path.with_suffix('.png')
    gif_path = obj_path.with_suffix('.gif')
    if image_path.exists() and gif_path.exists():
        return  # Skip rendering if already done

    mesh = trimesh.load(obj_path, force='mesh')

    # Set up scene
    scene = pyrender.Scene(bg_color=[1.0, 1.0, 1.0, 0.0])
    mesh_pyrender = pyrender.Mesh.from_trimesh(mesh, smooth=False)
    scene.add(mesh_pyrender)

    # Camera
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    cam_node = scene.add(camera, pose=np.eye(4))

    # Light
    light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
    scene.add(light, pose=np.eye(4))

    # Try to create Offscreen Renderer
    try:
        r = pyrender.OffscreenRenderer(viewport_width=image_size[0], viewport_height=image_size[1])
    except Exception as e:
        raise RuntimeError(f"Failed to create OffscreenRenderer: {e}")

    def create_camera_pose(angle_deg, distance=2.5):
        angle_rad = np.radians(angle_deg)
        eye = np.array([np.sin(angle_rad) * distance, 0.7 * distance, np.cos(angle_rad) * distance])
        center = np.array([0.0, 0.0, 0.0])
        up = np.array([0.0, 1.0, 0.0])

        z = (eye - center)
        z /= np.linalg.norm(z)
        x = np.cross(up, z)
        x /= np.linalg.norm(x)
        y = np.cross(z, x)
        y /= np.linalg.norm(y)

        pose = np.eye(4)
        pose[0:3, 0] = x
        pose[0:3, 1] = y
        pose[0:3, 2] = z
        pose[0:3, 3] = eye
        return pose

    # Render first image
    cam_pose = create_camera_pose(0)
    scene.set_pose(cam_node, pose=cam_pose)
    color, _ = r.render(scene)
    imageio.imwrite(image_path, color)

    # Generate frames for GIF
    frames = []
    for i in range(n_frames):
        angle = (360 / n_frames) * i
        cam_pose = create_camera_pose(angle)
        scene.set_pose(cam_node, pose=cam_pose)
        color, _ = r.render(scene)
        frames.append(color)

    # Save GIF
    imageio.mimsave(gif_path, frames, fps=10)

    r.delete()

def process_all_objs(folder_path, n_workers=4):
    folder_path = Path(folder_path)
    obj_files = list(folder_path.glob('*.obj'))

    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = {executor.submit(render_obj, obj_path): obj_path for obj_path in obj_files}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing .obj files"):
            obj_path = futures[future]
            try:
                future.result()
            except Exception as e:
                print(f"Error processing {obj_path}: {e}")


In [2]:
process_all_objs("/home/student/Documents/Data/Pre-processed_data/windows-preprocessed/synthetisch_0_v2/buis_recht/recht_bocht_15", n_workers=8)


NoSuchDisplayException: Cannot connect to "None"